## Dimensionality Reduction: Exercise Faces

### Setup

This project requires Python 3.7 or above:

In [2]:
import sys

assert sys.version_info >= (3, 7)

It also requires Scikit-Learn ≥ 1.0.1:

In [3]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

### The Olivetti Faces Dataset

*The classic Olivetti faces dataset contains 400 grayscale 64 × 64–pixel images of faces. Each image is flattened to a 1D vector of size 4,096. 40 different people were photographed (10 times each), and the usual task is to train a model that can predict which person is represented in each picture. Load the dataset using the `sklearn.datasets.fetch_olivetti_faces()` function.*

In [4]:
from sklearn.datasets import fetch_olivetti_faces

olivetti = fetch_olivetti_faces()

In [5]:
print(olivetti.DESCR)

.. _olivetti_faces_dataset:

The Olivetti faces dataset
--------------------------

`This dataset contains a set of face images`_ taken between April 1992 and 
April 1994 at AT&T Laboratories Cambridge. The
:func:`sklearn.datasets.fetch_olivetti_faces` function is the data
fetching / caching function that downloads the data
archive from AT&T.

.. _This dataset contains a set of face images: http://www.cl.cam.ac.uk/research/dtg/attarchive/facedatabase.html

As described on the original website:

    There are ten different images of each of 40 distinct subjects. For some
    subjects, the images were taken at different times, varying the lighting,
    facial expressions (open / closed eyes, smiling / not smiling) and facial
    details (glasses / no glasses). All the images were taken against a dark
    homogeneous background with the subjects in an upright, frontal position 
    (with tolerance for some side movement).

**Data Set Characteristics:**

    =================   ===========

In [6]:
olivetti.target

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22,
       22, 22, 22, 22, 22

We show the first 100 images together with their label. 

In [7]:
import matplotlib.pyplot as plt

plt.rc('font', size=8)
plt.rc('axes', labelsize=8, titlesize=8)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

def plot_faces(faces, labels, n_cols=10):
    faces = faces.reshape(-1, 64, 64)
    n_rows = (len(faces) - 1) // n_cols + 1
    plt.figure(figsize=(n_cols, n_rows * 1.1))
    for index, (face, label) in enumerate(zip(faces, labels)):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(face, cmap="gray")
        plt.axis("off")
        plt.title(label)
    plt.show()

plot_faces(olivetti.data[:100,:],olivetti.target[:100])

Split the dataset into a training set  and a test set (note that the dataset is already scaled between 0 and 1).

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(olivetti.data, olivetti.target, test_size=0.2, random_state=42)

Apply a RandomForestClassifier with 150 trees to classify the images.  
- Check the duaration of the training phase. 
- What's the accuracy on the test set? 

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=150, random_state=42)
%time rf_clf.fit(X_train, y_train)
rf_clf.score(X_test, y_test)

CPU times: total: 1.89 s
Wall time: 1.89 s


0.9125

Now apply PCA but keep 99% variance. 
- What's the percentage of the dimensions that are left? 
- Check the duaration of the training phase. 
- What's the accuracy on the test set? 
- Is this a dataset that takes advantage from PCA? 

In [16]:
X_train.shape

(320, 4096)

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(pca.n_components_)

print(pca.n_components_/X_train.shape[1])

223
0.054443359375


In [18]:
clf = RandomForestClassifier(n_estimators=150, random_state=42)
%time clf.fit(X_train_pca, y_train)
clf.score(X_test_pca, y_test)

CPU times: total: 484 ms
Wall time: 470 ms


0.9375

Find the best parameter combination using grid search. 
- Find the optimal number of pca components. 
- Find the optimal number of trees in the range 100 - 300 (with steps of 25)
- What's the accuracy? 

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import numpy as np

clf = make_pipeline(PCA(random_state=42), RandomForestClassifier(random_state=42))

param_grid = [{"pca__n_components": np.arange(50,220,10), 
               "randomforestclassifier__n_estimators": np.arange(100,300,25)}]

grid_search = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy')
%time grid_search.fit(X_train, y_train)

CPU times: total: 5min 34s
Wall time: 3min 14s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=42))]),
             param_grid=[{'pca__n_components': array([ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150, 160, 170,
       180, 190, 200, 210]),
                          'randomforestclassifier__n_estimators': array([100, 125, 150, 175, 200, 225, 250, 275])}],
             scoring='accuracy')

In [20]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'pca__n_components': 110, 'randomforestclassifier__n_estimators': 225}
0.8998412978310704


Retrain with the best parameters found. 
- Determine the training time and compare is with the original time without dimensionality reduction
- What's the possible use of this information if you spend much time finding the optimal parameter combination? 

In [21]:
pca = PCA(n_components=110)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

clf = RandomForestClassifier(n_estimators=225, random_state=42)

%time clf.fit(X_train_pca, y_train)
clf.score(X_test_pca, y_test)


CPU times: total: 531 ms
Wall time: 532 ms


0.9375

Could this technique be used for compression? Visualize the first 200 compressed images after recovery. 

In [22]:
pca = PCA(n_components=110)

olivetti_compressed = pca.fit_transform(olivetti.data[:200,:])


In [24]:
olivetti_compressed.shape

(200, 110)

In [25]:
olivetti_recovered = pca.inverse_transform(olivetti_compressed)
olivetti_recovered.shape

(200, 4096)

In [26]:
plot_faces(olivetti_recovered, olivetti.target[:200])

...